# Import

In [1]:
import pandas as pd
import numpy as np
import random

# att48

## Function

In [2]:
### Get points
def get_points(data='data/att48.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def df(list_gen, list_ant, list_distance, list_path):
    df = pd.DataFrame({'Gen' : list_gen, 'Ant' : list_ant,  'Distance' : list_distance, 'Path' : list_path})
    # min_distance = df[df.Distance == df.Distance.min()].Path.values[0]
    # for i in range(len(min_distance)):
    #     if i != len(paths)-1:
    #         t_new = ((1-p)*t0[min_distance[i]][min_distance[i+1]]) + (1/total_distance)
    #         t0[paths[i]][paths[i+1]] = t_new
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [3]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 10
ants= 48
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens = []
list_all_paths = []
list_all_total_distance = []
list_shortest_distance_value = []

list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_ants = []
    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_ants.append(ant)
        list_total_distance.update({ant : total_distance})
        list_paths.update({ant : paths})
        print(f'Total distance : {total_distance}')


#         print(f'list_gens : {list_gens}')
#         print(f'list_total_distance : {list_total_distance}')
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    ### Pair paths
    list_pair_paths = []
    for i in range(len(list_paths)):
        pair_paths = []
        for j in range(len(list_paths)):
            try:
                pair_paths.append([list_paths[i][j], list_paths[i][j+1]])
            except:
                pass
        list_pair_paths.append(pair_paths)
    
    ### Count paths for update pheromone
    count = {}
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            if not str(j) in count:
                count[str(j)] = 1
            else:
                count[str(j)] += 1

    list_count = []
    for i in list(count.keys()):
        i = i.split(',')
        count_0 = int(i[0].replace('[', ''))
        count_1 = int(i[1].replace(']', ''))
        list_count.append([count_0, count_1])

    for i in list_count:
        t0[i[0]][i[1]] += (1-p)*t0[i[0]][i[1]]
        # t0[i[1]][i[0]] += (1-p)*t0[i[1]][i[0]]

    ### Plus by summation walked path
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            t0[j[0]][j[1]] += 100/_distance
            # t0[j[1]][j[0]] += 1/_distance

            # print(f"t0[j[0]][j[1]] : {t0[j[0]][j[1]]}")
            # print(f"t0[j[1]][j[0]] : {t0[j[1]][j[0]]}")
            # print(j[0], j[1],distance)
            # print(j[1], j[0],distance)
    #         # print("------------------")

#     print(list_paths)
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # print(f'Shortest distance value: {shortest_distance_value}')
    # print(f'Shortest distance index: {shortest_distance_index}')
    # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
#     print(f'-------------')

    list_gens.append(list_ants)
    list_all_paths.append(list_paths)
    list_all_total_distance.append(list_total_distance)

    # for i in range(len(list_all_paths)):
    #     print(list_all_paths[i].values())

print(f'List of shortest distance values : {sorted(list_shortest_distance_value)}')

Gen : 0, Ant : 0
Paths : [20, 46, 10, 22, 13, 24, 12, 38, 31, 47, 4, 28, 1, 41, 9, 23, 44, 34, 25, 3, 33, 2, 21, 15, 0, 8, 37, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 45, 32, 14, 11, 19, 39, 7, 40]
Total distance : 38703.24564638667
Gen : 0, Ant : 1
Paths : [45, 32, 14, 11, 10, 22, 13, 24, 12, 20, 46, 19, 35, 17, 6, 27, 5, 36, 18, 26, 42, 29, 16, 43, 30, 37, 8, 0, 7, 21, 15, 2, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1, 39]
Total distance : 39726.799210855
Gen : 0, Ant : 2
Paths : [22, 10, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 46, 20, 12, 24, 13, 33, 2, 21, 15, 0, 8, 37, 30, 39, 7, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1]
Total distance : 38139.81128027202
Gen : 0, Ant : 3
Paths : [13, 24, 12, 22, 10, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 46, 20, 38, 31, 47, 4, 28, 1, 41, 9, 23, 44, 34, 25, 3, 33, 2, 21, 15, 0, 8, 37, 30, 39, 7, 40]
Total distance : 38948.8584939329
Gen : 0, Ant : 4
Paths : [9, 23

In [4]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43, 30, 37, 8, 0, 7, 21, 15, 2, 22, 10, 11, 14, 32, 45, 39, 19, 46, 20, 12, 24, 13, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1]': 35135.455097450664}

# berlin52

## Function

In [5]:
### Get points
def get_points(data='data/berlin52.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def df(list_gen, list_ant, list_distance, list_path):
    df = pd.DataFrame({'Gen' : list_gen, 'Ant' : list_ant,  'Distance' : list_distance, 'Path' : list_path})
    # min_distance = df[df.Distance == df.Distance.min()].Path.values[0]
    # for i in range(len(min_distance)):
    #     if i != len(paths)-1:
    #         t_new = ((1-p)*t0[min_distance[i]][min_distance[i+1]]) + (1/total_distance)
    #         t0[paths[i]][paths[i+1]] = t_new
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [6]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 52
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens = []
list_all_paths = []
list_all_total_distance = []
list_shortest_distance_value = []

list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_ants = []
    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_ants.append(ant)
        list_total_distance.update({ant : total_distance})
        list_paths.update({ant : paths})
        print(f'Total distance : {total_distance}')


#         print(f'list_gens : {list_gens}')
#         print(f'list_total_distance : {list_total_distance}')
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    ### Pair paths
    list_pair_paths = []
    for i in range(len(list_paths)):
        pair_paths = []
        for j in range(len(list_paths)):
            try:
                pair_paths.append([list_paths[i][j], list_paths[i][j+1]])
            except:
                pass
        list_pair_paths.append(pair_paths)
    
    ### Count paths for update pheromone
    count = {}
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            if not str(j) in count:
                count[str(j)] = 1
            else:
                count[str(j)] += 1

    list_count = []
    for i in list(count.keys()):
        i = i.split(',')
        count_0 = int(i[0].replace('[', ''))
        count_1 = int(i[1].replace(']', ''))
        list_count.append([count_0, count_1])

    for i in list_count:
        t0[i[0]][i[1]] += (1-p)*t0[i[0]][i[1]]
        # t0[i[1]][i[0]] += (1-p)*t0[i[1]][i[0]]

    ### Plus by summation walked path
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            t0[j[0]][j[1]] += 100/_distance
            # t0[j[1]][j[0]] += 1/_distance

            # print(f"t0[j[0]][j[1]] : {t0[j[0]][j[1]]}")
            # print(f"t0[j[1]][j[0]] : {t0[j[1]][j[0]]}")
            # print(j[0], j[1],distance)
            # print(j[1], j[0],distance)
    #         # print("------------------")

#     print(list_paths)
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # print(f'Shortest distance value: {shortest_distance_value}')
    # print(f'Shortest distance index: {shortest_distance_index}')
    # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
#     print(f'-------------')

    list_gens.append(list_ants)
    list_all_paths.append(list_paths)
    list_all_total_distance.append(list_total_distance)

    # for i in range(len(list_all_paths)):
    #     print(list_all_paths[i].values())

print(f'List of shortest distance values : {sorted(list_shortest_distance_value)}')

Gen : 0, Ant : 0
Paths : [17, 30, 21, 0, 48, 31, 35, 34, 33, 38, 39, 37, 36, 47, 23, 4, 14, 5, 3, 24, 45, 43, 15, 49, 19, 22, 29, 20, 16, 2, 18, 44, 40, 7, 9, 8, 42, 32, 50, 11, 27, 26, 25, 46, 12, 13, 51, 10, 28, 41, 6, 1]
Total distance of ant [0]: 8401.851818187733
Gen : 0, Ant : 1
Paths : [12, 26, 27, 25, 46, 13, 51, 10, 50, 11, 24, 3, 5, 4, 14, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 49, 19, 22, 30, 17, 21, 0, 48, 31, 44, 18, 40, 7, 9, 8, 42, 32, 2, 16, 20, 29, 28, 41, 6, 1]
Total distance of ant [1]: 8115.435584058902
Gen : 0, Ant : 2
Paths : [8, 9, 7, 40, 18, 44, 31, 48, 0, 21, 30, 17, 2, 16, 20, 22, 19, 49, 15, 43, 33, 34, 35, 38, 39, 37, 36, 47, 23, 4, 14, 5, 3, 24, 45, 28, 29, 41, 6, 1, 42, 32, 50, 11, 27, 26, 25, 46, 12, 13, 51, 10]
Total distance of ant [2]: 8028.519215682724
Gen : 0, Ant : 3
Paths : [9, 8, 7, 40, 18, 44, 31, 48, 0, 21, 30, 17, 2, 16, 20, 22, 19, 49, 15, 43, 33, 34, 35, 38, 39, 37, 36, 47, 23, 4, 14, 5, 3, 24, 45, 28, 29, 41, 6, 1, 42, 32, 50, 11, 2

In [7]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4, 14, 5, 3, 24, 11, 27, 26, 25, 46, 12, 13, 51, 10, 50, 32, 42, 9, 8, 7, 40, 18, 44, 31, 48, 0, 21, 30, 17, 2, 16, 20, 22, 19, 49, 15, 28, 29, 41, 6, 1]': 7310.669644787184,
 '[45, 43, 33, 34, 35, 38, 39, 37, 36, 47, 23, 4, 14, 5, 3, 24, 11, 27, 26, 25, 46, 12, 13, 51, 10, 50, 32, 42, 9, 8, 7, 40, 18, 44, 31, 48, 0, 21, 30, 17, 2, 16, 20, 22, 19, 49, 15, 28, 29, 41, 6, 1]': 7234.268534155313}

# eil76

## Function

In [8]:
### Get points
def get_points(data='data/eil76.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def df(list_gen, list_ant, list_distance, list_path):
    df = pd.DataFrame({'Gen' : list_gen, 'Ant' : list_ant,  'Distance' : list_distance, 'Path' : list_path})
    # min_distance = df[df.Distance == df.Distance.min()].Path.values[0]
    # for i in range(len(min_distance)):
    #     if i != len(paths)-1:
    #         t_new = ((1-p)*t0[min_distance[i]][min_distance[i+1]]) + (1/total_distance)
    #         t0[paths[i]][paths[i+1]] = t_new
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [9]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 76
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens = []
list_all_paths = []
list_all_total_distance = []
list_shortest_distance_value = []

list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_ants = []
    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_ants.append(ant)
        list_total_distance.update({ant : total_distance})
        list_paths.update({ant : paths})
        print(f'Total distance : {total_distance}')


#         print(f'list_gens : {list_gens}')
#         print(f'list_total_distance : {list_total_distance}')
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    ### Pair paths
    list_pair_paths = []
    for i in range(len(list_paths)):
        pair_paths = []
        for j in range(len(list_paths)):
            try:
                pair_paths.append([list_paths[i][j], list_paths[i][j+1]])
            except:
                pass
        list_pair_paths.append(pair_paths)
    
    ### Count paths for update pheromone
    count = {}
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            if not str(j) in count:
                count[str(j)] = 1
            else:
                count[str(j)] += 1

    list_count = []
    for i in list(count.keys()):
        i = i.split(',')
        count_0 = int(i[0].replace('[', ''))
        count_1 = int(i[1].replace(']', ''))
        list_count.append([count_0, count_1])

    for i in list_count:
        t0[i[0]][i[1]] += (1-p)*t0[i[0]][i[1]]
        # t0[i[1]][i[0]] += (1-p)*t0[i[1]][i[0]]

    ### Plus by summation walked path
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            t0[j[0]][j[1]] += 100/_distance
            # t0[j[1]][j[0]] += 1/_distance

            # print(f"t0[j[0]][j[1]] : {t0[j[0]][j[1]]}")
            # print(f"t0[j[1]][j[0]] : {t0[j[1]][j[0]]}")
            # print(j[0], j[1],distance)
            # print(j[1], j[0],distance)
    #         # print("------------------")

#     print(list_paths)
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # print(f'Shortest distance value: {shortest_distance_value}')
    # print(f'Shortest distance index: {shortest_distance_index}')
    # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
#     print(f'-------------')

    list_gens.append(list_ants)
    list_all_paths.append(list_paths)
    list_all_total_distance.append(list_total_distance)

    # for i in range(len(list_all_paths)):
    #     print(list_all_paths[i].values())

print(f'List of shortest distance values : {sorted(list_shortest_distance_value)}')

Gen : 0, Ant : 0
Paths : [5, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12, 26, 51, 44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 62, 15, 50, 16, 39, 11, 25, 3, 29, 1, 0, 42, 40, 41, 63, 21, 60, 68, 35, 70, 59, 69, 19, 36, 4, 14, 56, 58, 10, 64, 37, 9, 57, 71, 38, 8, 31, 43, 2, 48, 23, 17, 49, 24, 54, 30, 65, 22, 55]
Total distance : 644.7205042162967
Gen : 0, Ant : 1
Paths : [58, 13, 52, 34, 6, 7, 45, 33, 51, 26, 44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 62, 15, 50, 5, 67, 74, 75, 66, 25, 11, 39, 16, 2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, 18, 53, 12, 56, 14, 4, 36, 19, 69, 59, 70, 35, 68, 60, 21, 0, 42, 40, 41, 63, 55, 22, 48, 23, 17, 49, 24, 54, 30, 3, 29, 1]
Total distance : 585.8878507518207
Gen : 0, Ant : 2
Paths : [21, 61, 72, 32, 62, 15, 50, 5, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12, 26, 51, 44, 28, 47, 46, 20, 73, 27, 1, 29, 3, 25, 11, 39, 16, 2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, 58, 56, 14, 4, 36, 19, 69, 59, 70, 35, 68, 60, 63, 41, 40, 42, 0, 22, 55

In [10]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[55, 22, 62, 15, 50, 5, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12, 26, 51, 44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 0, 42, 40, 41, 63, 21, 60, 68, 35, 70, 59, 69, 19, 36, 4, 14, 56, 3, 29, 1, 16, 39, 11, 25, 57, 71, 38, 8, 31, 43, 2, 48, 23, 17, 49, 24, 54, 30, 9, 37, 64, 10, 65, 58]': 571.1464908173365,
 '[55, 22, 48, 23, 17, 49, 24, 54, 30, 9, 37, 64, 10, 65, 58, 13, 18, 53, 12, 26, 51, 44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 62, 15, 50, 5, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 57, 71, 38, 8, 31, 43, 2, 39, 16, 25, 11, 3, 29, 1, 0, 42, 40, 41, 63, 21, 60, 68, 35, 70, 59, 69, 19, 36, 4, 14, 56]': 570.5691651199385}

# kroA100

## Function

In [11]:
### Get points
def get_points(data='data/kroA100.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def df(list_gen, list_ant, list_distance, list_path):
    df = pd.DataFrame({'Gen' : list_gen, 'Ant' : list_ant,  'Distance' : list_distance, 'Path' : list_path})
    # min_distance = df[df.Distance == df.Distance.min()].Path.values[0]
    # for i in range(len(min_distance)):
    #     if i != len(paths)-1:
    #         t_new = ((1-p)*t0[min_distance[i]][min_distance[i+1]]) + (1/total_distance)
    #         t0[paths[i]][paths[i+1]] = t_new
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [12]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 100
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens = []
list_all_paths = []
list_all_total_distance = []
list_shortest_distance_value = []

list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_ants = []
    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_ants.append(ant)
        list_total_distance.update({ant : total_distance})
        list_paths.update({ant : paths})
        print(f'Total distance : {total_distance}')


#         print(f'list_gens : {list_gens}')
#         print(f'list_total_distance : {list_total_distance}')
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    ### Pair paths
    list_pair_paths = []
    for i in range(len(list_paths)):
        pair_paths = []
        for j in range(len(list_paths)):
            try:
                pair_paths.append([list_paths[i][j], list_paths[i][j+1]])
            except:
                pass
        list_pair_paths.append(pair_paths)
    
    ### Count paths for update pheromone
    count = {}
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            if not str(j) in count:
                count[str(j)] = 1
            else:
                count[str(j)] += 1

    list_count = []
    for i in list(count.keys()):
        i = i.split(',')
        count_0 = int(i[0].replace('[', ''))
        count_1 = int(i[1].replace(']', ''))
        list_count.append([count_0, count_1])

    for i in list_count:
        t0[i[0]][i[1]] += (1-p)*t0[i[0]][i[1]]
        # t0[i[1]][i[0]] += (1-p)*t0[i[1]][i[0]]

    ### Plus by summation walked path
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            t0[j[0]][j[1]] += 100/_distance
            # t0[j[1]][j[0]] += 1/_distance

            # print(f"t0[j[0]][j[1]] : {t0[j[0]][j[1]]}")
            # print(f"t0[j[1]][j[0]] : {t0[j[1]][j[0]]}")
            # print(j[0], j[1],distance)
            # print(j[1], j[0],distance)
    #         # print("------------------")

#     print(list_paths)
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # print(f'Shortest distance value: {shortest_distance_value}')
    # print(f'Shortest distance index: {shortest_distance_index}')
    # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
#     print(f'-------------')

    list_gens.append(list_ants)
    list_all_paths.append(list_paths)
    list_all_total_distance.append(list_total_distance)

    # for i in range(len(list_all_paths)):
    #     print(list_all_paths[i].values())

print(f'List of shortest distance values : {sorted(list_shortest_distance_value)}')

Gen : 0, Ant : 0
Paths : [67, 84, 38, 29, 95, 77, 51, 4, 36, 32, 75, 12, 94, 81, 49, 43, 72, 68, 63, 39, 53, 1, 80, 24, 60, 50, 86, 8, 6, 56, 19, 11, 26, 85, 34, 61, 59, 76, 22, 97, 90, 44, 31, 10, 14, 16, 73, 20, 58, 71, 9, 83, 35, 37, 23, 17, 78, 52, 87, 15, 21, 93, 69, 65, 64, 3, 96, 55, 79, 30, 88, 41, 7, 91, 0, 62, 5, 48, 89, 18, 74, 25, 98, 46, 92, 27, 66, 57, 54, 82, 33, 28, 45, 2, 42, 13, 70, 40, 99, 47]
Total distance : 24732.88406198989
Gen : 0, Ant : 1
Paths : [53, 39, 63, 68, 72, 49, 43, 1, 81, 94, 12, 32, 75, 36, 4, 77, 51, 95, 38, 29, 47, 99, 70, 40, 13, 2, 45, 28, 33, 82, 54, 11, 26, 85, 34, 19, 56, 6, 8, 86, 50, 60, 24, 80, 67, 84, 42, 61, 59, 76, 22, 97, 90, 44, 31, 10, 14, 16, 73, 20, 58, 71, 9, 83, 35, 37, 23, 17, 78, 52, 87, 15, 21, 93, 69, 65, 64, 3, 96, 55, 79, 30, 88, 41, 7, 91, 0, 62, 5, 48, 89, 18, 74, 25, 98, 46, 92, 27, 66, 57]
Total distance : 25188.583760453712
Gen : 0, Ant : 2
Paths : [70, 40, 99, 47, 13, 2, 45, 28, 33, 82, 54, 11, 26, 85, 34, 19, 56, 6, 8

In [13]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37, 35, 98, 83, 9, 71, 20, 73, 58, 16, 14, 10, 31, 90, 97, 22, 44, 46, 62, 5, 48, 89, 78, 52, 18, 74, 96, 55, 79, 30, 88, 41, 7, 91, 0, 92, 27, 66, 57, 60, 24, 80, 68, 72, 49, 43, 1, 53, 39, 63, 67, 84, 38, 29, 95, 77, 51, 4, 36, 32, 75, 12, 94, 81, 47, 99, 70, 40, 13, 2, 45, 28, 33, 82, 54, 11, 26, 85, 34, 19, 56, 6, 8, 86, 50, 76, 59, 61, 42]': 23137.514597468155,
 '[43, 49, 72, 67, 84, 38, 29, 95, 77, 51, 4, 36, 32, 75, 12, 94, 81, 1, 53, 39, 63, 68, 80, 24, 60, 50, 86, 8, 6, 56, 19, 11, 26, 85, 34, 61, 59, 76, 22, 97, 90, 44, 31, 10, 14, 16, 73, 20, 58, 71, 9, 83, 35, 37, 23, 17, 78, 52, 87, 15, 21, 93, 69, 65, 64, 3, 96, 55, 79, 30, 88, 41, 7, 91, 0, 62, 5, 48, 89, 18, 74, 25, 98, 46, 92, 27, 66, 57, 54, 82, 33, 28, 45, 2, 42, 13, 70, 40, 99, 47]': 24169.64503404339}

# pr76

## Function

In [14]:
### Get points
def get_points(data='data/pr76.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def df(list_gen, list_ant, list_distance, list_path):
    df = pd.DataFrame({'Gen' : list_gen, 'Ant' : list_ant,  'Distance' : list_distance, 'Path' : list_path})
    # min_distance = df[df.Distance == df.Distance.min()].Path.values[0]
    # for i in range(len(min_distance)):
    #     if i != len(paths)-1:
    #         t_new = ((1-p)*t0[min_distance[i]][min_distance[i+1]]) + (1/total_distance)
    #         t0[paths[i]][paths[i+1]] = t_new
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [15]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 76
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens = []
list_all_paths = []
list_all_total_distance = []
list_shortest_distance_value = []

list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_ants = []
    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_ants.append(ant)
        list_total_distance.update({ant : total_distance})
        list_paths.update({ant : paths})
        print(f'Total distance : {total_distance}')


#         print(f'list_gens : {list_gens}')
#         print(f'list_total_distance : {list_total_distance}')
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    ### Pair paths
    list_pair_paths = []
    for i in range(len(list_paths)):
        pair_paths = []
        for j in range(len(list_paths)):
            try:
                pair_paths.append([list_paths[i][j], list_paths[i][j+1]])
            except:
                pass
        list_pair_paths.append(pair_paths)
    
    ### Count paths for update pheromone
    count = {}
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            if not str(j) in count:
                count[str(j)] = 1
            else:
                count[str(j)] += 1

    list_count = []
    for i in list(count.keys()):
        i = i.split(',')
        count_0 = int(i[0].replace('[', ''))
        count_1 = int(i[1].replace(']', ''))
        list_count.append([count_0, count_1])

    for i in list_count:
        t0[i[0]][i[1]] += (1-p)*t0[i[0]][i[1]]
        # t0[i[1]][i[0]] += (1-p)*t0[i[1]][i[0]]

    ### Plus by summation walked path
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            t0[j[0]][j[1]] += 100/_distance
            # t0[j[1]][j[0]] += 1/_distance

            # print(f"t0[j[0]][j[1]] : {t0[j[0]][j[1]]}")
            # print(f"t0[j[1]][j[0]] : {t0[j[1]][j[0]]}")
            # print(j[0], j[1],distance)
            # print(j[1], j[0],distance)
    #         # print("------------------")

#     print(list_paths)
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # print(f'Shortest distance value: {shortest_distance_value}')
    # print(f'Shortest distance index: {shortest_distance_index}')
    # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
#     print(f'-------------')

    list_gens.append(list_ants)
    list_all_paths.append(list_paths)
    list_all_total_distance.append(list_total_distance)

    # for i in range(len(list_all_paths)):
    #     print(list_all_paths[i].values())

print(f'List of shortest distance values : {sorted(list_shortest_distance_value)}')

Gen : 0, Ant : 0
Paths : [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Total distance : 125691.9989350622
Gen : 0, Ant : 1
Paths : [62, 63, 56, 57, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 10, 11, 12, 13, 14, 15, 9, 8, 5, 6, 7, 2, 3, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 48, 49, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 25, 26, 32, 31, 37, 38, 60, 61, 71, 72, 70, 69, 73]
Total distance : 135097.73149684575
Gen : 0, Ant : 2
Paths : [23, 24, 20, 21, 22, 0, 1, 2, 3, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 57, 56, 62, 63, 61, 60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 10, 11, 12, 13, 14, 15, 9, 8, 5, 6, 7, 73, 37, 38, 32, 31, 25, 26, 43, 47, 46, 44, 45, 68, 67, 66, 4

In [16]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 57, 56, 62, 63, 61, 60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 10, 11, 12, 13, 14, 15, 9, 8, 5, 6, 7, 2, 3, 4, 19, 18, 30, 29, 28, 25, 26, 43, 47, 46, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73, 37, 38, 48, 49, 66, 67, 68, 69, 70, 71, 72]': 123389.07045571336}

# rd100

## Function

In [19]:
### Get points
def get_points(data='data/rd100.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(float(list_data[i].split(' ')[1])), int(float(list_data[i].split(' ')[2]))))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def df(list_gen, list_ant, list_distance, list_path):
    df = pd.DataFrame({'Gen' : list_gen, 'Ant' : list_ant,  'Distance' : list_distance, 'Path' : list_path})
    # min_distance = df[df.Distance == df.Distance.min()].Path.values[0]
    # for i in range(len(min_distance)):
    #     if i != len(paths)-1:
    #         t_new = ((1-p)*t0[min_distance[i]][min_distance[i+1]]) + (1/total_distance)
    #         t0[paths[i]][paths[i+1]] = t_new
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [20]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 100
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens = []
list_all_paths = []
list_all_total_distance = []
list_shortest_distance_value = []

list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_ants = []
    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_ants.append(ant)
        list_total_distance.update({ant : total_distance})
        list_paths.update({ant : paths})
        print(f'Total distance : {total_distance}')


#         print(f'list_gens : {list_gens}')
#         print(f'list_total_distance : {list_total_distance}')
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    ### Pair paths
    list_pair_paths = []
    for i in range(len(list_paths)):
        pair_paths = []
        for j in range(len(list_paths)):
            try:
                pair_paths.append([list_paths[i][j], list_paths[i][j+1]])
            except:
                pass
        list_pair_paths.append(pair_paths)
    
    ### Count paths for update pheromone
    count = {}
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            if not str(j) in count:
                count[str(j)] = 1
            else:
                count[str(j)] += 1

    list_count = []
    for i in list(count.keys()):
        i = i.split(',')
        count_0 = int(i[0].replace('[', ''))
        count_1 = int(i[1].replace(']', ''))
        list_count.append([count_0, count_1])

    for i in list_count:
        t0[i[0]][i[1]] += (1-p)*t0[i[0]][i[1]]
        # t0[i[1]][i[0]] += (1-p)*t0[i[1]][i[0]]

    ### Plus by summation walked path
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            t0[j[0]][j[1]] += 100/_distance
            # t0[j[1]][j[0]] += 1/_distance

            # print(f"t0[j[0]][j[1]] : {t0[j[0]][j[1]]}")
            # print(f"t0[j[1]][j[0]] : {t0[j[1]][j[0]]}")
            # print(j[0], j[1],distance)
            # print(j[1], j[0],distance)
    #         # print("------------------")

#     print(list_paths)
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # print(f'Shortest distance value: {shortest_distance_value}')
    # print(f'Shortest distance index: {shortest_distance_index}')
    # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
#     print(f'-------------')

    list_gens.append(list_ants)
    list_all_paths.append(list_paths)
    list_all_total_distance.append(list_total_distance)

    # for i in range(len(list_all_paths)):
    #     print(list_all_paths[i].values())

print(f'List of shortest distance values : {sorted(list_shortest_distance_value)}')

Gen : 0, Ant : 0
Paths : [99, 43, 28, 34, 95, 38, 54, 39, 6, 33, 40, 21, 4, 60, 10, 35, 56, 46, 98, 52, 5, 78, 9, 32, 2, 3, 31, 77, 19, 73, 25, 8, 11, 84, 13, 81, 74, 20, 48, 66, 96, 62, 85, 14, 61, 17, 0, 59, 7, 68, 89, 50, 82, 67, 70, 86, 12, 63, 90, 64, 79, 80, 29, 47, 41, 42, 24, 23, 51, 44, 15, 97, 65, 75, 57, 87, 83, 30, 22, 1, 88, 58, 94, 76, 92, 27, 36, 18, 45, 49, 72, 55, 91, 26, 16, 71, 69, 37, 53, 93]
Total distance : 9452.31118731814
Gen : 0, Ant : 1
Paths : [66, 96, 62, 85, 14, 61, 17, 0, 59, 7, 68, 89, 50, 82, 67, 70, 86, 13, 84, 11, 81, 74, 20, 48, 12, 3, 31, 77, 19, 73, 25, 8, 32, 9, 78, 5, 52, 98, 46, 56, 35, 63, 90, 64, 79, 80, 29, 47, 21, 40, 33, 6, 39, 41, 42, 24, 23, 54, 38, 95, 28, 34, 43, 99, 51, 44, 15, 97, 65, 75, 57, 87, 83, 30, 22, 1, 88, 58, 94, 76, 92, 27, 36, 18, 45, 49, 72, 55, 91, 26, 16, 71, 69, 37, 53, 2, 93, 10, 60, 4]
Total distance : 9370.967756854807
Gen : 0, Ant : 2
Paths : [68, 7, 59, 0, 17, 61, 86, 14, 62, 96, 85, 66, 12, 48, 20, 74, 81, 84, 11,

In [ ]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43, 30, 37, 8, 0, 7, 21, 15, 2, 22, 10, 11, 14, 32, 45, 39, 19, 46, 20, 12, 24, 13, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1]': 35135.455097450664}